This is the first method of Approach 2 for Transient Requirement validation (Section 4.2.2). Users need to read and run [Transient_base.ipynb](./Transient_base.ipynb) before reading and runing this notebook.

In approach 2.1, the number of pixel pairs which meet the mission requirement as a percentage of the total number of pixel pairs selected are counted.

The first method we apply to evaluate the noise structure is similar to that in Section 4.2.1, except that the percentage of measurements that fall below the threshold curve is calculated for each of the 5-km-wide bins. If the average of the percentages from all bins is larger than 0.683, we judge that the noise level falls below the requirement. 

In [1]:
import numpy as np
import pickle
from pathlib import Path
import pandas as pd

In [2]:
calval_dir = Path.cwd()/'calval'
calval_location = 'central_valley'
# calval_location = 'texas'
# calval_location = 'oklahoma'
# calval_location = 'purtorico'
work_dir = calval_dir/calval_location

In [3]:
with open(work_dir/'approach2.pkl','rb') as f:
    dist,rel_measure, ifgs_date = pickle.load(f)

In [4]:
n_ifgs = len(dist)

In [5]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [6]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [7]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

In [8]:
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

In [9]:
columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

In [10]:
index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

In [11]:
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points in each bin:

In [12]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,911,2660,4328,5648,7096,8494,9772,10643,11739,12428,61291
20190203-20190215,910,2704,4253,5662,7065,8481,9696,10602,11876,12541,61249
20190227-20190311,925,2606,4225,5698,7111,8487,9607,10796,11546,12477,61001
20190323-20190404,877,2644,4266,5754,7060,8487,9699,10666,11694,12564,61147
20190416-20190428,927,2610,4213,5619,7085,8481,9732,10811,11616,12467,61094
20190510-20190522,952,2703,4324,5725,7236,8357,9541,10702,11775,12668,61315
20190603-20190615,926,2621,4248,5785,7210,8601,9779,10805,11589,12539,61564
20190627-20190709,935,2612,4309,5881,7211,8611,9559,10703,11823,12561,61644
20190721-20190802,917,2607,4294,5665,7243,8460,9534,10737,11765,12491,61222
20190814-20190826,962,2586,4280,5789,7059,8545,9628,10631,11534,12493,61014


Number of data points that below the curve:

In [13]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,788,2255,3702,4848,6108,7405,8589,9325,10343,10991,53363
20190203-20190215,652,1959,3228,4382,5573,6708,7712,8458,9537,10068,48209
20190227-20190311,728,1967,3185,4236,5202,6187,6868,7648,8028,8682,44049
20190323-20190404,783,2337,3677,4913,5900,7109,8004,8770,9608,10341,51101
20190416-20190428,728,1979,3153,4176,5254,6389,7227,8028,8665,9420,45599
20190510-20190522,709,2060,3271,4252,5472,6337,7379,8339,9210,10037,47029
20190603-20190615,806,2300,3554,4689,5695,6718,7339,8023,8414,8854,47538
20190627-20190709,854,2284,3607,4711,5633,6511,7141,7947,8706,9393,47394
20190721-20190802,839,2341,3723,4936,6293,7244,8302,9391,10407,11110,53476
20190814-20190826,843,2177,3390,4440,5266,6231,6866,7328,7876,8498,44417


Ratio of pass:

In [14]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total,mean
20190110-20190122,0.864984,0.847744,0.855360,0.858357,0.860767,0.871792,0.878940,0.876163,0.881080,0.884374,0.870650,0.867956
20190203-20190215,0.716484,0.724482,0.758994,0.773931,0.788818,0.790944,0.795380,0.797774,0.803048,0.802807,0.787099,0.775266
20190227-20190311,0.787027,0.754797,0.753846,0.743419,0.731543,0.728997,0.714895,0.708411,0.695306,0.695840,0.722103,0.731408
20190323-20190404,0.892816,0.883888,0.861932,0.853841,0.835694,0.837634,0.825240,0.822239,0.821618,0.823066,0.835707,0.845797
20190416-20190428,0.785329,0.758238,0.748398,0.743193,0.741567,0.753331,0.742602,0.742577,0.745954,0.755595,0.746374,0.751678
20190510-20190522,0.744748,0.762116,0.756475,0.742707,0.756219,0.758286,0.773399,0.779200,0.782166,0.792311,0.767006,0.764763
20190603-20190615,0.870410,0.877528,0.836629,0.810545,0.789875,0.781072,0.750486,0.742527,0.726033,0.706117,0.772172,0.789122
20190627-20190709,0.913369,0.874426,0.837085,0.801054,0.781168,0.756126,0.747045,0.742502,0.736361,0.747791,0.768834,0.793693
20190721-20190802,0.914940,0.897967,0.867024,0.871315,0.868839,0.856265,0.870778,0.874639,0.884573,0.889440,0.873477,0.879578
20190814-20190826,0.876299,0.841841,0.792056,0.766972,0.745998,0.729198,0.713128,0.689305,0.682851,0.680221,0.727980,0.751787


Percentage of interferograms passes the requirement (0.683):

In [15]:
np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs

1.0

In [16]:
np.count_nonzero(ratio_pd['mean']>thresthod)/n_ifgs

1.0